In [2]:
import pandas as pd
import numpy as np

In [3]:
books = pd.read_csv('csv/BX-Books.csv', error_bad_lines=False, sep=';')
books.drop(['Image-URL-S', 'Image-URL-M', 'Image-URL-L'], axis=1, inplace=True)

users = pd.read_csv('csv/BX-Users.csv', error_bad_lines=False, sep=';')

ratings = pd.read_csv('csv/BX-Book-Ratings.csv', error_bad_lines=False, sep=';')


b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'


In [8]:
books['Year-Of-Publication'].unique()

array([2002, 2001, 1991, 1999, 2000, 1993, 1996, 1988, 2004, 1998, 1994,
       2003, 1997, 1983, 1979, 1995, 1982, 1985, 1992, 1986, 1978, 1980,
       1952, 1987, 1990, 1981, 1989, 1984, 0, 1968, 1961, 1958, 1974,
       1976, 1971, 1977, 1975, 1965, 1941, 1970, 1962, 1973, 1972, 1960,
       1966, 1920, 1956, 1959, 1953, 1951, 1942, 1963, 1964, 1969, 1954,
       1950, 1967, 2005, 1957, 1940, 1937, 1955, 1946, 1936, 1930, 2011,
       1925, 1948, 1943, 1947, 1945, 1923, 2020, 1939, 1926, 1938, 2030,
       1911, 1904, 1949, 1932, 1928, 1929, 1927, 1931, 1914, 2050, 1934,
       1910, 1933, 1902, 1924, 1921, 1900, 2038, 2026, 1944, 1917, 1901,
       2010, 1908, 1906, 1935, 1806, 2021, '2000', '1995', '1999', '2004',
       '2003', '1990', '1994', '1986', '1989', '2002', '1981', '1993',
       '1983', '1982', '1976', '1991', '1977', '1998', '1992', '1996',
       '0', '1997', '2001', '1974', '1968', '1987', '1984', '1988',
       '1963', '1956', '1970', '1985', '1978', '1973', '1980'

In [10]:
books.loc[books.ISBN == '0789466953','Year-Of-Publication'] = 2000
books.loc[books.ISBN == '0789466953','Book-Author'] = "James Buckley"
books.loc[books.ISBN == '0789466953','Publisher'] = "DK Publishing Inc"
books.loc[books.ISBN == '0789466953','Book-Title'] = "DK Readers: Creating the X-Men, How Comic Books Come to Life (Level 4: Proficient Readers)"
books.loc[books.ISBN == '078946697X','Year-Of-Publication'] = 2000
books.loc[books.ISBN == '078946697X','Book-Author'] = "Michael Teitelbaum"
books.loc[books.ISBN == '078946697X','Publisher'] = "DK Publishing Inc"
books.loc[books.ISBN == '078946697X','Book-Title'] = "DK Readers: Creating the X-Men, How It All Began (Level 4: Proficient Readers)"
books.loc[books.ISBN == '2070426769','Year-Of-Publication'] = 2003
books.loc[books.ISBN == '2070426769','Book-Author'] = "Jean-Marie Gustave Le ClÃ?Â©zio"
books.loc[books.ISBN == '2070426769','Publisher'] = "Gallimard"
books.loc[books.ISBN == '2070426769','Book-Title'] = "Peuple du ciel, suivi de 'Les Bergers"

books['Year-Of-Publication']=pd.to_numeric(books['Year-Of-Publication'], errors='coerce')
books.loc[(books['Year-Of-Publication'] > 2020) | (books['Year-Of-Publication'] == 0),'Year-Of-Publication'] = np.NAN
books['Year-Of-Publication'].fillna(round(books['Year-Of-Publication'].mean()), inplace=True)
books['Year-Of-Publication'] = books['Year-Of-Publication'].astype(np.int32)

In [11]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [12]:
users.loc[(users['Age'] > 100) | (users['Age'] < 5), 'Age'] = np.nan
users['Age'] = users['Age'].fillna(users['Age'].mean())
users['Age'] = users['Age'].astype(np.int32)


In [13]:
ratings_new = ratings[ratings['ISBN'].isin(books['ISBN'])]

In [14]:
n_users = users.shape[0]
n_books = books.shape[0]
1.0-len(ratings_new)/float(n_users*n_books)

0.9999863734155897

In [15]:
ratings_implicit = ratings_new[ratings_new['Book-Rating'] == 0]
ratings_explicit = ratings_new[ratings_new['Book-Rating'] != 0]


In [17]:
counts1 = ratings_explicit['User-ID'].value_counts()
ratings_explicit = ratings_explicit[ratings_explicit['User-ID'].isin(counts1[counts1 >= 15].index)]
counts = ratings_explicit['Book-Rating'].value_counts()
ratings_explicit = ratings_explicit[ratings_explicit['Book-Rating'].isin(counts[counts >= 15].index)]

In [18]:
ratings_matrix = ratings_explicit.pivot(index='User-ID', columns='ISBN', values='Book-Rating')
userID = ratings_matrix.index
ISBN = ratings_matrix.columns
print(ratings_matrix.shape)
ratings_matrix

(4458, 114047)


ISBN,0000913154,0001046438,000104687X,0001047213,0001047973,000104799X,0001048082,0001053736,0001053744,0001055607,...,B0000T6KIM,B0000VZEH8,B0000VZEJQ,B0000X8HIE,B00011SOXI,B00013AX9E,B0001FZGRQ,B0001GMSV2,B0001I1KOG,B000234N3A
User-ID,,,,,,,,,,,,,,,,,,,,,
242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
388,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
503,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278356,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278582,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
ratings_matrix.fillna(0, inplace = True)
ratings_matrix = ratings_matrix.astype(np.int32)
ratings_matrix

ISBN,0000913154,0001046438,000104687X,0001047213,0001047973,000104799X,0001048082,0001053736,0001053744,0001055607,...,B0000T6KIM,B0000VZEH8,B0000VZEJQ,B0000X8HIE,B00011SOXI,B00013AX9E,B0001FZGRQ,B0001GMSV2,B0001I1KOG,B000234N3A
User-ID,,,,,,,,,,,,,,,,,,,,,
242,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
243,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
254,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
388,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
503,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278356,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
278418,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
278582,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [119]:
import math

def get_cosine_cor(series1, series2):
    correlation = 0
    size1 = 0
    size2 = 0
    for i in range(0, len(series1)):
        correlation += series1[i] * series2[i]
        size1 += series1[i] * series1[i]
        size2 += series2[i] * series2[i]
    size1 = math.sqrt(size1)
    size2 = math.sqrt(size2)
    return correlation / (size1 * size2)

In [76]:
import random

def get_user_similarity(rates, other_user, method = 'pearson'):
    other_rates = ratings_matrix.loc[other_user]
    if method == 'cosine':
        return get_cosine_cor(rates, other_rates)
    else:
        return rates.corr(other_rates, method='pearson')

def get_not_rated_books(rates, other_user):
    other_rates = ratings_matrix.loc[other_user]
    ISBNs = ratings_matrix.columns
    not_rateds = []
    for i in range(0, len(other_rates)):
        if rates[i] == 0 and other_rates[i] > 0:
            book_rate = (ISBNs[i], other_rates[i])
            not_rateds.append(book_rate)
    return not_rateds

def predict_rates_cf(user_rates, method='pearson'):
    users = ratings_matrix.index
    similar_user = users[0]
    score_max = get_user_similarity(user_rates, similar_user, method)
    for user in users:
        score = get_user_similarity(user_rates, user, method)
        if score > score_max:
            score_max = score
            similar_user = user
    
    not_rateds = get_not_rated_books(user_rates, similar_user)
    if len(not_rateds) > 0:
        book_index = random.randint(0, len(not_rateds))
        return not_rateds[book_index][0]
    else:
        return None

def predict_rates_cf_id(user_id, method='pearson'):
    user_rates = ratings_matrix.loc[user_id] 
    return predict_rates_cf(user_rates, method)

In [122]:
#book prediction example (not working for now - function doesn't finish)
predicted_book = predict_rates_cf_id(242, 'cosine')
print(predicted_book)

KeyboardInterrupt: 

In [121]:
#just for test
user_rates = ratings_matrix.loc[242]
similarity = get_user_similarity(user_rates, 13850, 'pearson')
print(similarity)

0.022887746497261167


In [110]:
#just for test
def find_similar_users():
    users = ratings_matrix.index
    users_count = len(users)
    for i in range(0, users_count):
        rates_1 = ratings_matrix.loc[users[i]]
        if rates_1.sum() > 15:
            for j in range(i + 1, users_count):
                rates_2 = ratings_matrix.loc[users[j]]
                if rates_2.sum() > 15:
                    for k in range(0, len(rates_2)):
                        if rates_1[k] == rates_2[k] and rates_1[k] > 0:
                            return users[i], users[j]

similar_users = find_similar_users()
print(similar_users)

(242, 13850)
